In [69]:
import pandas as pd
import numpy as np
from forex_python.converter import CurrencyRates
import re
from datetime import datetime
import openpyxl

In [70]:
data = pd.read_csv('analyticsAssignmentData.csv',na_values=np.NAN)

## 1 EXPLORING THE DATASET

In [71]:
data.head()

,eventTimestamp,userID,sessionID,platform,userCountry,realCurrencyAmount,realCurrencyAmount.1,realCurrencyType
0,2019-11-1 0:36:27,16707,NaN,ANDROID,NaN,1998,19.98,AUD
1,2019-11-1 1:37:4,25142,c13fd09a-cce2-419f-9f1e-533258c19d61,ANDROID,US,499,4.99,USD
2,2019-11-1 1:37:46,25142,c13fd09a-cce2-419f-9f1e-533258c19d61,ANDROID,US,499,4.99,USD
3,2019-11-1 2:16:25,101530,NaN,ANDROID,NaN,1249,12.49,USD
4,2019-11-1 2:58:7,102640,6fe2173d-923b-4590-b1a3-797904672d11,ANDROID,US,499,4.99,USD


In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 8 columns):
eventTimestamp          3000 non-null object
userID                  3000 non-null int64
sessionID               2862 non-null object
platform                3000 non-null object
userCountry             2862 non-null object
realCurrencyAmount      3000 non-null int64
realCurrencyAmount.1    3000 non-null float64
realCurrencyType        1000 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 187.6+ KB


## 2 FEATURE MANIPULATION

In [73]:
data.eventTimestamp = data.eventTimestamp.astype(np.datetime64)
data.userID = data.userID.astype(np.object_)

In [74]:
data.drop('realCurrencyAmount',axis=1,inplace=True)

In [75]:
data.rename({'realCurrencyAmount.1':'realCurrencyAmount'},axis=1,inplace=True)

In [76]:
data.describe(exclude='number')

,eventTimestamp,userID,sessionID,platform,userCountry,realCurrencyType
count,3000,3000.0,2862,3000,2862,1000
unique,2811,522.0,966,2,26,15
top,2019-11-26 12:02:26,20057.0,b948f791-6c95-4d9b-9686-1339c1d4540e,IOS,US,USD
freq,4,328.0,35,1696,1244,462
first,2019-11-01 00:36:27,NaN,NaN,NaN,NaN,NaN
last,2019-11-30 23:50:24,NaN,NaN,NaN,NaN,NaN


In [77]:
data.describe()

,realCurrencyAmount
count,3000.000000
mean,9.694010
std,66.165425
min,0.000000
25%,0.000000
50%,0.000000
75%,4.990000
max,1590.000000


### 2.1 FILLING NA VALUES

In [78]:
countryCode = ['US', 'DK', 'CA', 'CH', 'FR',
               'DE', 'GB', 'NL', 'SE', 'AU',
               'ES', 'BE', 'RU', 'AT', 'GR',
               'NO', 'IE', 'BY', 'BR', 'NZ',
               'LV','JP']
countryCurrency = ['USD', 'DKK', 'CAD', 'CHF', 'EUR',
                   'EUR', 'GBP', 'EUR', 'SEK', 'AUD',
                   'EUR', 'EUR', 'RUB', 'EUR', 'USD',
                   'NOK', 'EUR', 'USD', 'BRL', 'NZD',
                   'EUR', 'JPY']

In [79]:
for i, j in zip(countryCode,countryCurrency):
    data.loc[(data.realCurrencyType.isna()) & (data.userCountry == i),'realCurrencyType'] = j

In [80]:
for i, j in zip(countryCode,countryCurrency):
    data.loc[(data.realCurrencyType == j) & (data.userCountry.isna()),'userCountry'] = i

In [81]:
data.loc[(data.realCurrencyType == 'TWD') & (data.userCountry.isna()),'userCountry'] = 'TW'
data.loc[(data.realCurrencyType == 'THB') & (data.userCountry.isna()),'userCountry'] = 'TH'

In [82]:
print(data.userCountry.nunique())
print(np.sort(data.userCountry.unique()))

27
['AT' 'AU' 'BE' 'BR' 'BY' 'CA' 'CH' 'DE' 'DK' 'ES' 'FI' 'FR' 'GB' 'GR'
 'HU' 'IE' 'IT' 'JP' 'LV' 'NL' 'NO' 'NZ' 'RU' 'SE' 'TH' 'TW' 'US']


In [83]:
code = {'AT':'EUR', 'AU':'AUD', 'BE':'EUR', 'BR':'BRL', 'BY':'USD',
        'CA':'CAD', 'CH':'CHF', 'DE':'EUR', 'DK':'DKK', 'ES':'EUR',
        'FI':'EUR', 'FR':'EUR', 'GB':'GBP', 'GR':'USD', 'HU':'HUF',
        'IE':'EUR', 'IT':'EUR', 'JP':'JPY', 'LV':'EUR', 'NL':'EUR',
        'NO':'NOK', 'NZ':'NZD', 'RU':'RUB', 'SE':'SEK', 'TH':'THB',
        'TW':'TWD', 'US':'USD'}

In [84]:
data.loc[:,'realCurrencyType'] = data.userCountry.map(code)

## 3 CREATING NEW COLUMN FOR CONVERSION TO USD

In [85]:
c = CurrencyRates()

In [86]:
date = data.eventTimestamp.astype(np.str_)

In [87]:
data['usdCurrency'] = np.NaN

In [88]:
# for i in np.arange(0, len(data.index)):
#     base = data.loc[i, 'realCurrencyType']
#     if(base != 'TWD'):
#         amount = data.loc[i, 'realCurrencyAmount']
#         date1 = re.findall(r'[0-9]+',date[i])
#         date1 = list(map(int, date1))
#         usd = c.convert(base, 'USD', amount, datetime(date1[0],date1[1],date1[2],date1[3],date1[4],date1[5]))
#         data.loc[i,'usdCurrency'] = np.round(usd,2)
# for i in np.arange(0, 3000):
#     base = data.loc[i, 'realCurrencyType']
#     if(base != 'TWD'):
#         amount = data.loc[i, 'realCurrencyAmount']
#         date1 = re.findall(r'[0-9]+',date[i])
#         date1 = list(map(int, date1))
#         usd = c.convert(base, 'USD', amount, datetime(date1[0],date1[1],date1[2],date1[3],date1[4],date1[5]))
#         print(i,'=',np.round(usd, decimals=2))

In [89]:
data.loc[data.realCurrencyType == 'TWD',:]

,eventTimestamp,userID,sessionID,platform,userCountry,realCurrencyAmount,realCurrencyType,usdCurrency
104,2019-11-02 10:41:04,17188,NaN,IOS,TW,400.0,TWD,NaN
435,2019-11-07 13:56:21,18933,NaN,ANDROID,TW,410.0,TWD,NaN
1619,2019-11-23 12:26:22,36483,NaN,ANDROID,TW,410.0,TWD,NaN


**Not gonna make loop for only three rows**
- **rate of taiwan dollar on 2, 7 and 23 were same 0.033**

In [90]:
data.loc[(data.realCurrencyType == 'TWD') & (data.platform == 'IOS'),'usdCurrency'] = np.round(400*0.033,2)
data.loc[(data.realCurrencyType == 'TWD') & (data.platform == 'ANDROID'),'usdCurrency'] = np.round(410*0.033,2)

### saving into csv file so we can restore for analysis

In [91]:
# data.to_csv('newData.csv',index=False)

In [92]:
data = pd.read_csv('newData.csv',na_values=np.NaN)

---

#### **Now Data is ready for Analysis of Questions asked:**
- **Q1. D1 retention, D7 retention, D28 retention (both classic and rolling)**
- **Q2. Daily active users**
- **Q3. ARPDAU**
- **Q4. Maximum revenue earned by a user**
- **Q5. Average sessions/user**
- **Q6. Total users in the game**

### Q1. D1, D7, and D28 Classic And Rolling Retention
- CLASSIC RETENTION

In [93]:
data['date'] = date.apply(lambda x: re.search(r'2019-11-[0-9][0-9]?',x).group())
data['date'] = data.date.astype(np.str_)

In [94]:
list1 = list(data.loc[data.date == '2019-11-01','userID'].unique())

In [95]:
count = {}
count['d1'] = 0
for i in data.loc[data.date == '2019-11-02','userID'].unique():
    if(i in list1):
        count['d1'] = count['d1'] + 1

In [96]:
count['d7'] = 0
for i in data.loc[data.date == '2019-11-07','userID'].unique():
    if(i in list1):
        count['d7'] = count['d7'] + 1

In [97]:
count['d28'] = 0
for i in data.loc[data.date == '2019-11-28','userID'].unique():
    if(i in list1):
        count['d28'] = count['d28'] + 1

In [98]:
classicRetention = {}
classicRetention['d1'] = (7/40)*100
classicRetention['d7'] = (3/40)*100
classicRetention['d28'] = (1/40)*100

In [99]:
print(data.loc[data.date == '2019-11-01','userID'].nunique())
print(count['d1'])
print(count['d7'])
print(count['d28'])

40
7
3
1


In [100]:
print('Day 1 Classic Retention: ',classicRetention['d1'])
print('Day 7 Classic Retention: ',classicRetention['d7'])
print('Day 28 Classic Retention: ',classicRetention['d28'])

Day 1 Classic Retention:  17.5
Day 7 Classic Retention:  7.5
Day 28 Classic Retention:  2.5


- ROLLING RETENTION

In [101]:
a = (data.date == '2019-11-01')
b = ((data.date == '2019-11-01')|(data.date == '2019-11-02')|(data.date == '2019-11-03')|
     (data.date == '2019-11-04')|(data.date == '2019-11-05')|(data.date == '2019-11-06'))
c = (data.date == '2019-11-28')
count['d1'] = 0
for i in data.loc[-a,'userID'].unique():
    if(i in list1):
        count['d1'] = count['d1'] + 1
count['d7'] = 0
for i in data.loc[-b,'userID'].unique():
    if(i in list1):
        count['d7'] = count['d7'] + 1
count['d28'] = 0
for i in data.loc[c,'userID'].unique():
    if(i in list1):
        count['d28'] = count['d28'] + 1

In [102]:
print(count['d1'])
print(count['d7'])
print(count['d28'])

20
10
1


In [103]:
rollingRetention = {}
rollingRetention['d1'] = (20/40)*100
rollingRetention['d7'] = (10/40)*100
rollingRetention['d28'] = (1/40)*100

In [104]:
print('Day 1 Rolling Retention: ',rollingRetention['d1'])
print('Day 7 Rolling Retention: ',rollingRetention['d7'])
print('Day 28 Rolling Retention: ',rollingRetention['d28'])

Day 1 Rolling Retention:  50.0
Day 7 Rolling Retention:  25.0
Day 28 Rolling Retention:  2.5


### Q2. Daily Active Users

In [105]:
print('Daily Active Users\n')
print('Days','\t\t','No. of users')
for i in data.loc[:,'date'].unique():
    print(i,'\t',data.loc[data.date == i,'userID'].nunique())

Daily Active Users

Days 		 No. of users
2019-11-01 	 40
2019-11-02 	 29
2019-11-03 	 34
2019-11-04 	 22
2019-11-05 	 29
2019-11-06 	 23
2019-11-07 	 25
2019-11-08 	 23
2019-11-09 	 29
2019-11-10 	 32
2019-11-11 	 25
2019-11-12 	 19
2019-11-13 	 23
2019-11-14 	 21
2019-11-15 	 22
2019-11-16 	 17
2019-11-17 	 20
2019-11-18 	 16
2019-11-19 	 16
2019-11-20 	 14
2019-11-21 	 17
2019-11-22 	 36
2019-11-23 	 44
2019-11-24 	 34
2019-11-25 	 36
2019-11-26 	 25
2019-11-27 	 29
2019-11-28 	 38
2019-11-29 	 53
2019-11-30 	 49


### Q3. ARPDAU

In [106]:
print('Average Revenue Per Daily Active User\n')
print('Days','\t\t','ARPDAU','\t','No. of users','\t','usdRevenue')
for i in data.loc[:,'date'].unique():
    a = data.loc[data.date == i,'userID'].nunique()
    b = np.round(data.loc[data.date == i,'usdCurrency'].sum(),2)
    c = np.round(b/a,2)
    print(i,'\t',c,'\t\t',a,'\t\t',b)

Average Revenue Per Daily Active User

Days 		 ARPDAU 	 No. of users 	 usdRevenue
2019-11-01 	 18.14 		 40 		 725.46
2019-11-02 	 12.69 		 29 		 367.9
2019-11-03 	 16.14 		 34 		 548.81
2019-11-04 	 13.99 		 22 		 307.81
2019-11-05 	 27.59 		 29 		 800.21
2019-11-06 	 12.63 		 23 		 290.5
2019-11-07 	 12.5 		 25 		 312.57
2019-11-08 	 14.33 		 23 		 329.51
2019-11-09 	 14.54 		 29 		 421.74
2019-11-10 	 14.23 		 32 		 455.4
2019-11-11 	 17.4 		 25 		 434.89
2019-11-12 	 9.8 		 19 		 186.17
2019-11-13 	 10.27 		 23 		 236.32
2019-11-14 	 12.0 		 21 		 251.92
2019-11-15 	 8.86 		 22 		 195.02
2019-11-16 	 12.74 		 17 		 216.53
2019-11-17 	 13.43 		 20 		 268.53
2019-11-18 	 16.42 		 16 		 262.73
2019-11-19 	 10.28 		 16 		 164.46
2019-11-20 	 12.34 		 14 		 172.71
2019-11-21 	 13.62 		 17 		 231.6
2019-11-22 	 5.43 		 36 		 195.52
2019-11-23 	 6.89 		 44 		 303.35
2019-11-24 	 12.17 		 34 		 413.85
2019-11-25 	 6.49 		 36 		 233.71
2019-11-26 	 9.33 		 25 		 233.17
2019-11-27 	 10.12 		 

In [107]:
data.loc[data.date == '2019-11-01','usdCurrency'].sum()

725.46

### Q4. Maximum Earned By A User

In [108]:
userMax = pd.pivot_table(data,values='usdCurrency',index=['userID'],aggfunc=np.sum)
userMax.reset_index(inplace=True)

In [109]:
userMax.loc[userMax.usdCurrency == userMax.usdCurrency.max(),:]

,userID,usdCurrency
200,104340,549.78


### Q5. Average Sessions Per User

In [110]:
count = 0
for i in np.sort(data[data.sessionID.isna()]['userID'].unique()):
    booloo = data.loc[data.userID == i,'sessionID'].notna().any()
    if(booloo != True):
        count = count + 1
print(count)

75


In [111]:
uniqueSessions = count + data.sessionID.nunique()
avgSessionsPerUser = uniqueSessions / data.userID.nunique()
print('Average Sessions Per User: ', avgSessionsPerUser)

Average Sessions Per User:  1.9942528735632183


### Q6. Total Number Of Users In The Game

In [44]:
print('Total Number Of Users In The Game: ', data.userID.nunique())

Total Number Of Users In The Game:  522


In [45]:
# data.to_excel('finalAnalysis.xlsx',index=False)